# Threading Methods
Allows parallel optuna run

In [120]:
# import
import optuna
from datetime import datetime
from threading import Thread

CPU = 16
TIMEOUT=10

In [121]:
from time import sleep


class Objective(object):
    def __init__(self, min_x, max_x):
        # Hold this implementation specific arguments as the fields of the class.
        self.min_x = min_x
        self.max_x = max_x

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        x = trial.suggest_float("x", self.min_x, self.max_x)
        return (x - 2) ** 2

In [122]:
def make_study(name="ml_clustering"):
    now = datetime.now()
    studyname = name + now.strftime("%Y-%m-%d_%H%M%S")
    study = optuna.create_study(study_name=studyname, storage="sqlite:///clustering.db")
    print("Created study "+study.study_name)

    # fixed_params = {key: val for key, val in [("q", Q), ("eps", EPS)] if val is not None}
    # study.sampler = optuna.samplers.PartialFixedSampler(fixed_params, study.sampler)
    return studyname

studyname = make_study()

[I 2022-05-05 03:55:30,120] A new study created in RDB with name: ml_clustering2022-05-05_035529


Created study ml_clustering2022-05-05_035529


In [123]:
def singlethread_optimize(timeout):
    study = optuna.load_study(study_name=studyname, storage="sqlite:///clustering.db")
    study.optimize(Objective(-100, 100), timeout=timeout)

# singlethread_optimize(TIMEOUT)

In [124]:
def multithread_optimize(timeout, cpu=CPU):
    print("Optimization with {} threads.".format(cpu))

    processes = []
    for i in range(cpu):
        processes.append(Thread(target=singlethread_optimize, args=(timeout,), name="Thread_{}".format(i)))
        processes[-1].start()

    for proc in processes:
        proc.join()
        print("{} finished!".format(proc.name))

    print("Optimization finished!")

multithread_optimize(TIMEOUT)

Optimization with 16 threads.


[I 2022-05-05 03:55:31,786] Trial 0 finished with value: 3645.5648515369835 and parameters: {'x': -58.37851316103257}. Best is trial 0 with value: 2680.529296104523.
[I 2022-05-05 03:55:32,934] Trial 5 finished with value: 2680.529296104523 and parameters: {'x': -49.77382829291768}. Best is trial 5 with value: 2680.529296104523.
[I 2022-05-05 03:55:32,984] Trial 4 finished with value: 4591.543181035882 and parameters: {'x': -65.76092665420008}. Best is trial 5 with value: 2680.529296104523.
[I 2022-05-05 03:55:33,077] Trial 7 finished with value: 1172.6506550480713 and parameters: {'x': 36.24398713713214}. Best is trial 7 with value: 54.61992594975748.
[I 2022-05-05 03:55:33,227] Trial 8 finished with value: 54.61992594975748 and parameters: {'x': 9.390529476956132}. Best is trial 8 with value: 54.61992594975748.
[I 2022-05-05 03:55:33,248] Trial 10 finished with value: 5882.653304013511 and parameters: {'x': 78.69845698587105}. Best is trial 8 with value: 54.61992594975748.
[I 2022-05

Thread_0 finished!
Thread_1 finished!
Thread_2 finished!
Thread_3 finished!
Thread_4 finished!
Thread_5 finished!
Thread_6 finished!
Thread_7 finished!
Thread_8 finished!
Thread_9 finished!
Thread_10 finished!
Thread_11 finished!
Thread_12 finished!
Thread_13 finished!
Thread_14 finished!
Thread_15 finished!
Optimization finished!


In [125]:
def optimize_results():
    study = optuna.load_study(study_name=studyname, storage="sqlite:///clustering.db")
    optuna.visualization.plot_slice(study).show()
    optuna.visualization.plot_contour(study, params=["x"]).show()
    # optuna.visualization.plot_contour(study, params=["q", "eps"]).show()
    optuna.visualization.plot_param_importances(study).show()
    # return study.best_trial.params["eps"], study.best_trial.params["q"]

optimization_res = optimize_results()

[W 2022-05-05 03:55:41,841] The length of params must be greater than 1.
